In [1]:
import pandas as pd
import glob
import re
file_paths = glob.glob(r'1984 - 2022 멜론 Top.100\\*.멜론TOP100가사.csv')  # 데이터 위치 확인 필요
df_list = [pd.read_csv(file_path,index_col=0) for file_path in file_paths]
combined_df = pd.concat(df_list, ignore_index=True)
combined_df.rename(columns={'가사':'lyrics','제목':'title','장르':'genre'},inplace=True)
combined_df.drop_duplicates(ignore_index=True,inplace=True)
combined_df = combined_df[(combined_df['lyrics']!='Nan')&(combined_df['title']!='NaN') ]
combined_df.dropna(inplace=True)
combined_df['lyrics'] =combined_df['lyrics'].apply(lambda x: re.sub(r'\n+', '\n',x))
combined_df['lyrics'] =combined_df['lyrics'].apply(lambda x: re.sub(r'\s*\n\s*', '\n',x))
combined_df['lyrics'] =combined_df['lyrics'].apply(lambda x: re.sub(r'\s*!\n*', '\n',x))
combined_df['lyrics'] =combined_df['lyrics'].apply(lambda x: re.sub(r'\s*\n', '\n',x))
combined_df['lyrics'] =combined_df['lyrics'].apply(lambda x: re.sub(r'\n\s*', '\n',x))
combined_df['lyrics'] = combined_df['lyrics'].str.replace('  ', ' ')
combined_df['lyrics'] = combined_df['lyrics'].str.strip()
df_1,df_2,df_3,df_4,df_5,df_6 = (combined_df.copy(deep=True) for _ in range(6))

In [2]:
#단어단위 뭉치 NL줄 단위\n",
def word_N_split(lyrics_row,l_num):
    l_num =l_num
    word_sum = 0
    lyrics_mung = []
    lyrics_word =[]
    for word in lyrics_row.split('\n'): #df_6['lyrics'].iloc[1]
        word_sum += count_char(word)
        if word_sum>l_num:
            lyrics_mung.append(lyrics_word)
            word_sum = 0
            lyrics_word=[]
            word_sum+=count_char(word)
        lyrics_word.append(word)
    if word_sum<=l_num:
        lyrics_mung.append(lyrics_word)
    return [[word for phrase in sublist for word in phrase.split() if word !=''] for sublist in lyrics_mung]

#한줄단위 뭉치 NL줄 단위
def line_N_split(lyrics_row,l_num):
    word_sum = 0
    lyrics_mung = []
    lyrics_line =[]
    for line in lyrics_row.split('\n'): #df_6['lyrics'].iloc[1]
        word_sum += count_char(line)

        if word_sum>l_num:
            lyrics_mung.append(lyrics_line)
            word_sum = 0
            lyrics_line=[]
            word_sum += count_char(line)
        lyrics_line.append(line)
    if word_sum<=l_num:
        lyrics_mung.append(lyrics_line)
    return lyrics_mung
# 글자수만 count
def count_char(text):
    return sum(1 for char in text if char.isalpha())

In [3]:
# label syllable_limit마다
n_ = 1
df_1['label'] = df_1['lyrics'].apply(lambda x: [' '.join(x.split('\n')[i:i+n_]).split(' ') for i in range(0, len(x.split('\n')), n_) if x.split('\n')[i:i+n_]!=''])
df_1['chunk_strategy'] = 'word'
df_1['sampling_strategy'] ='one'
df_1['syllable_limit'] = 0
df_1['mungchi'] = df_1['label'].apply(lambda x: [[count_char(word) for word in sublist] for sublist in x])
# label 전곡
df_2['label'] = df_2['lyrics'].apply(lambda x: [[word for part in x.split('\n') for word in part.split(' ') if word]])
df_2['chunk_strategy'] = 'word'
df_2['sampling_strategy'] ='total'
df_2['syllable_limit'] = 0
df_2['mungchi'] = df_2['label'].apply(lambda x: [[count_char(word) for word in sublist] for sublist in x])

l_num = 30
df_3['label'] = df_3['lyrics'].apply(lambda x: word_N_split(x,l_num))
df_3['chunk_strategy'] = 'word'
df_3['sampling_strategy'] ='L'
df_3['syllable_limit'] = l_num
df_3['mungchi'] = df_3['label'].apply(lambda x: [[count_char(word) for word in sublist] for sublist in x])
# 한label syllable_limit
n_ = 1
df_4['label'] = df_4['lyrics'].apply(lambda x: [x.strip().split('\n')[i:i+n_] for i in range(0, len(x.strip().split('\n')), n_)])
df_4['chunk_strategy'] = 'line'
df_4['sampling_strategy'] ='one'
df_4['syllable_limit'] = 0
df_4['mungchi'] = df_4['label'].apply(lambda x: [[count_char(word) for word in sublist] for sublist in x])
# 한label 전곡
df_5['label']= df_5['lyrics'].apply(lambda x: [x.strip().split('\n')])
df_5['chunk_strategy'] = 'line'
df_5['sampling_strategy'] ='total'
df_5['syllable_limit'] = 0
df_5['mungchi'] = df_5['label'].apply(lambda x: [[count_char(word.strip()) for word in sublist] for sublist in x])

df_6['label'] = df_6['lyrics'].apply(lambda x: line_N_split(x,l_num))
df_6['chunk_strategy'] = 'line'
df_6['sampling_strategy'] ='L'
df_6['syllable_limit'] = l_num
df_6['mungchi'] = df_6['label'].apply(lambda x: [[count_char(word) for word in sublist] for sublist in x])

In [4]:
dataset = pd.concat([df_1, df_2, df_3, df_4, df_5, df_6], axis=0)
dataset = dataset[['title','lyrics','genre','mungchi','label','chunk_strategy','sampling_strategy','syllable_limit']]

In [ ]:
# data 저장
dataset.to_csv('lyrics_dataset.csv',index_label=False,encoding='utf-8-sig')
dataset.reset_index().to_json('lyrics_dataset.json')

### DATA 검증

In [5]:
# 리스트안에 빈칸 있는 가사 찾기\n",
def zero_check(df):
    idx = 0
    chk = 0
    for i in df.label:
        idx+=1
        for j in i:
            for x in j:
                if len(x)==0:
                    chk += 1
                    print(idx-1,len(x),i)
    if chk ==0:
        return True
    else:
        return False
    
# 리스트안에 특정문자 있는 가사 찾기
def word_check(df,word):
    idx = 0
    for i in df.label:
        idx+=1
        for j in i:
            for x in j:
                if x==word:
                    print(idx-1,i)
    return

# 이중리스트 여부 체크
def is_double_list(df):
    chk =0
    idx=0
    for i in df.label:
        idx+=1
        for j in i:
            if isinstance(j,list)==False:
                chk+=1
                print(idx,j)
    idx=0                
    for i in df.mungchi:
        idx+=1
        for j in i:
            if isinstance(j,list)==False:
                chk+=1
                print(idx,j)
    if chk==0:
        return True
    else:       
        return False

In [8]:
zero_check(dataset)

True

In [9]:
is_double_list(dataset)

True

In [10]:
# 랜덤 샘플 육안 확인
import random
i = random.randint(1, len(df_1)+1)

print('____type_1')
print(df_1[['chunk_strategy','sampling_strategy','syllable_limit']].iloc[i].tolist())
print(df_1['label'].iloc[i])
print(df_1['mungchi'].iloc[i])
print('____type_2')
print(df_2[['chunk_strategy','sampling_strategy','syllable_limit']].iloc[i].tolist())
print(df_2['label'].iloc[i])
print(df_2['mungchi'].iloc[i])
print('____type_3')
print(df_3[['chunk_strategy','sampling_strategy','syllable_limit']].iloc[i].tolist())
print(df_3['label'].iloc[i])
print(df_3['mungchi'].iloc[i])
print('____type_4')
print(df_4[['chunk_strategy','sampling_strategy','syllable_limit']].iloc[i].tolist())
print(df_4['label'].iloc[i])
print(df_4['mungchi'].iloc[i])
print('____type_5')
print(df_5[['chunk_strategy','sampling_strategy','syllable_limit']].iloc[i].tolist())
print(df_5['label'].iloc[i])
print(df_5['mungchi'].iloc[i])
print('____type_6')
print(df_6[['chunk_strategy','sampling_strategy','syllable_limit']].iloc[i].tolist())
print(df_6['label'].iloc[i])
print(df_6['mungchi'].iloc[i])

____type_1
['word', 'one', 0]
[['슬피', '날', '안아준', '아침', '햇살이'], ['소란히', '날', '스친', '실바람결이'], ['내', '눈', '속에', '살아가려'], ['날', '찾는', '밤하늘', '별도', '그댈', '테죠'], ['닿지', '못해서', '볼', '수', '없어'], ['사무치게', '늘', '안고', '싶어서'], ['바람이', '실어준', '향기'], ['가슴으로', '안는', '날'], ['어디서', '숨어보는가요'], ['또', '숨고', '숨어도', '또', '엇갈려', '가도'], ['내', '나이만큼에', '첫눈이', '내려와'], ['모든', '걸', '지워도', '내', '슬픈', '가슴은'], ['항상', '그대', '찾을', '테니'], ['다시', '사랑할', '그날까지만', '안녕'], ['착한', '그댄', '지금', '어디', '있는지'], ['행여', '내', '생각에', '울지는', '않는지'], ['이런', '날', '알고도', '볼', '수', '없는'], ['그대', '마음이', '더', '아플', '텐데'], ['긴', '밤', '지나면', '아침', '오고'], ['비', '온', '뒤에', '꽃들이', '자라고'], ['계절이', '돌아오듯이'], ['달도', '밤을', '찾듯이'], ['모두', '다', '제자릴', '찾겠죠'], ['또', '숨고', '숨어도', '또', '엇갈려가도'], ['내', '나이만큼에', '첫눈이', '내려와'], ['모든', '걸', '지워도', '내', '슬픈', '가슴은'], ['항상', '그대', '찾을', '테니'], ['다시', '사랑할', '그날까지만', '우리', '안녕'], ['그땐', '내가', '더', '사랑할', '테니'], ['운명이', '우리를'], ['다시', '허락할', '그날까지만', '안녕'], ['꼭', '기억해줘요', '그날까지만', '안녕']]
[[2, 1, 3, 2, 3], [3, 1